## Import statements

In [1]:
# Standard libraries
import sys
import glob
import os

# 3rd party libraries
import numpy as np
from pathlib import Path
from natsort import natsorted
from joblib import Parallel, delayed

# OME Zarr
import zarr
from dask.array import from_zarr, moveaxis
from numcodecs import Blosc
from ome_zarr.writer import write_image
from ome_zarr.io import parse_url

# Our libraries
import tif_to_ome
from cellAnalysis.cell_detection import createShardedPointAnnotation, readSectionTif
import ng_utils
import cors_webserver

## Functions

In [2]:
def convert_tif_to_zarr(img_dir, out_dir):
    """ Given an image directory and an output directory, 
    this function converts the tiff files into a zarr group.

    Args:
        img_dir (_type_): a string representing the path to the image directory.
        out_dir (_type_): a string representing the path to the output directory.

    Returns:
        _type_: a integer value of 0 if the conversion is successful, else 1
    """
    try:
        # List of image files that needs to be stacked.
        fns = natsorted(glob.glob(img_dir + "/*.tif", recursive=True))
        
        # Load the tiff directories
        print(fns)

        if not os.path.isdir(out_dir):
            os.mkdir(out_dir)

        # Path to zarr group
        zarr_path = os.path.join(out_dir, "data.zarr")

        # Create an ome zarr group
        ome_path = os.path.join(out_dir, "ome.zarr")
        image = tif_to_ome.readTifSection(str(fns[0]))
        w,h = image.shape

        # Write into zarr group
        zarr_file = zarr.open(zarr_path,shape=(len(fns), w, h), chunks = (1, w, h),  dtype='u2')
        Parallel(n_jobs=5, verbose=13)(delayed(tif_to_ome.readTifWrapper)(i, zarr_file, fn) for i, fn in enumerate(fns))
        print(zarr_file)

        # Read into a dask array
        dask_arr = from_zarr(zarr_file)
        print(dask_arr.shape)

        # Write to a ome Zarr group
        compressor = Blosc(cname="zstd", clevel=5, shuffle=Blosc.SHUFFLE)
        store = parse_url(ome_path, mode="w").store
        zarr_grp = zarr.open(store=store)
        z = 5
        w = 500
        h = 500
        #write_image(dask_arr, group = zarr_grp,axes="zxy",storage_options=dict(chunks=(z, w, h)))
        write_image(dask_arr, group=zarr_grp, axes="zyx", storage_options=dict(chunks=(z, w, h), compressor=compressor))

        print("OME zarr conversion done!")
        return 0
    except Exception as e:
        print(e)
        return 1
    
    
def convert_tif_to_zarr_rgb(img_dir_ch0, img_dir_ch1, img_dir_ch2, out_dir):
    """ Given an image directory and an output directory, 
    this function converts the tiff files into a zarr group.

    Args:
        img_dir (_type_): a string representing the path to the image directory.
        out_dir (_type_): a string representing the path to the output directory.

    Returns:
        _type_: a integer value of 0 if the conversion is successful, else 1
    """
    try:
        # List of image files that needs to be stacked.
        fns0 = natsorted(glob.glob(img_dir_ch0 + "/*.tif", recursive=True))[:5]
        fns1 = natsorted(glob.glob(img_dir_ch1 + "/*.tif", recursive=True))[:5]
        fns2 = natsorted(glob.glob(img_dir_ch2 + "/*.tif", recursive=True))[:5]
        
        assert len(fns0) == len(fns1) == len(fns2), "Mismatch in number of files between channels"

        # Load the tiff directories
        if not os.path.isdir(out_dir):
            os.mkdir(out_dir)

        # Path to zarr group
        zarr_path = os.path.join(out_dir, "data.zarr")

        # Create an ome zarr group
        ome_path = os.path.join(out_dir, "ome.zarr")
        image = tif_to_ome.readTifSection(str(fns0[0]))
        w,h = image.shape

        # Write into zarr group
        zarr_file = zarr.open(zarr_path,shape=(len(fns0), w, h, 3), chunks = (1, w, h, 3),  dtype='u2')
        
        def read_and_stack(i, zarr_file, fn0, fn1, fn2):
            img0 = tif_to_ome.readTifSection(fn0)
            img1 = tif_to_ome.readTifSection(fn1)
            img2 = tif_to_ome.readTifSection(fn2)
            rgb_image = np.stack((img0, img1, img2), axis=-1)
            zarr_file[i, :, :, :] = rgb_image
        
        Parallel(n_jobs=5, verbose=13)(delayed(read_and_stack)(i, zarr_file, fn0, fn1, fn2) for i, (fn0, fn1, fn2) in enumerate(zip(fns0, fns1, fns2)))
        print(zarr_file)

        # Read into a dask array
        dask_arr = from_zarr(zarr_file)
        print(dask_arr.shape)
        dask_arr = moveaxis(dask_arr, -1, 0)
        print(dask_arr.shape)

        # Write to a ome Zarr group
        compressor = Blosc(cname="zstd", clevel=5, shuffle=Blosc.SHUFFLE)
        store = parse_url(ome_path, mode="w").store
        zarr_grp = zarr.open(store=store)
        z = 5
        w = 500
        h = 500
        #write_image(dask_arr, group = zarr_grp,axes="zxy",storage_options=dict(chunks=(z, w, h)))
        write_image(dask_arr, group=zarr_grp, axes="czyx", storage_options=dict(chunks=(3, z, w, h), compressor=compressor))

        print("OME zarr conversion done!")
        return 0
    except Exception as e:
        print(e)
        return 1

### Run script

In [3]:
GENERATE = False

In [4]:
# Convert to ome zarr 
input_path = "../B0039_stitched/stitched_ch2/"
ome_zarr_output_path = "../B0039_stitched/stitched_counted/output/B0039_ome.zarr_2"

# Convert them to ome.zarr directories
if GENERATE:
    convert_tif_to_zarr(input_path, ome_zarr_output_path)

In [5]:
# Convert to ome zarr  RGB
input_path0 = "../B0039_stitched/stitched_ch0/"
input_path1 = "../B0039_stitched/stitched_ch1/"
input_path2 = "../B0039_stitched/stitched_ch2/"
ome_zarr_output_path = "../B0039_stitched/stitched_counted/output/B0039_rgb_ome.zarr"

# Convert them to ome.zarr directories
if GENERATE:
    convert_tif_to_zarr_rgb(input_path0, input_path1, input_path2, ome_zarr_output_path)

In [6]:
# Convert points to neuroglancer shards
inputpoints_path = "../B0039_stitched/stitched_counted/output/inputpoints.txt"
ng_points_path = "../B0039_stitched/stitched_counted/output/"

# Convert points to neuroglancer format
if GENERATE:
    inputpoints = np.loadtxt(inputpoints_path, skiprows=2)
    createShardedPointAnnotation(inputpoints, ng_points_path)

In [7]:
# Start up local server


# Generate neuroglancer URL
# https://xulabtexera.anat.uci.edu/

# Open up neuroglancer URL in browser and visualize


In [12]:
from ng_utils import ng_SingleSectionLocalViewer
import webbrowser


# Taken from cell detection preview code
section = 120
imgfiles = natsorted(glob.glob(os.path.join(input_path0, "*.tif"), recursive=True))
print(imgfiles)
print(section)

imgfile  = imgfiles[section]
image = readSectionTif(imgfile)
print("Reading cell data")

cells = np.loadtxt(inputpoints_path, skiprows=2)
cells = cells[cells[:, 0] == section, 1:]
print(cells)

viewer = ng_SingleSectionLocalViewer(image, cells)
webbrowser.open(viewer.get_viewer_url())
#input("Press any key to continue")
#viewer = ng_SingleSectionLocalViewer(image, cells, viewer)
#webbrowser.open(viewer.get_viewer_url())
#input("Press any key to exit")


['../B0039_stitched/stitched_ch0\\230628_B0039_PG_U01_280-0001_1_0.tif', '../B0039_stitched/stitched_ch0\\230628_B0039_PG_U01_280-0002_1_0.tif', '../B0039_stitched/stitched_ch0\\230628_B0039_PG_U01_280-0003_1_0.tif', '../B0039_stitched/stitched_ch0\\230628_B0039_PG_U01_280-0004_1_0.tif', '../B0039_stitched/stitched_ch0\\230628_B0039_PG_U01_280-0005_1_0.tif', '../B0039_stitched/stitched_ch0\\230628_B0039_PG_U01_280-0006_1_0.tif', '../B0039_stitched/stitched_ch0\\230628_B0039_PG_U01_280-0007_1_0.tif', '../B0039_stitched/stitched_ch0\\230628_B0039_PG_U01_280-0008_1_0.tif', '../B0039_stitched/stitched_ch0\\230628_B0039_PG_U01_280-0009_1_0.tif', '../B0039_stitched/stitched_ch0\\230628_B0039_PG_U01_280-0010_1_0.tif', '../B0039_stitched/stitched_ch0\\230628_B0039_PG_U01_280-0011_1_0.tif', '../B0039_stitched/stitched_ch0\\230628_B0039_PG_U01_280-0012_1_0.tif', '../B0039_stitched/stitched_ch0\\230628_B0039_PG_U01_280-0013_1_0.tif', '../B0039_stitched/stitched_ch0\\230628_B0039_PG_U01_280-0014_1

True

In [60]:
os.path.join(input_path0, "*.tif")

'../B0039_stitched/stitched_ch0/*.tif'

### Notes

In [ ]:
# Taken from cell detection preview code
def main():
    args = arg_parser().parse_args()
    img_dir = Path(args.img_dir)
    channel = args.channel
    section = args.section
    threshold  = args.threshold


    input_points_file = img_dir/"inputpoints.txt"

    imgfiles = natsorted(glob.glob(os.path.join(img_dir,"**/*1_{}.tif".format(channel)), recursive=True))
    print(img_dir/'**/*1_{}.tif'.format(channel))
    print(section)
    if section==-1:
        cells = Parallel(n_jobs=-4, verbose=13)(delayed(get_cell_locations)(img_file, intensity_threshold=threshold, index =i) for i, img_file in enumerate(imgfiles))
        points = np.vstack(cells)
        np.savetxt(input_points_file, points , "%d %d %d", header = "point\n"+str(cells.shape[0]), comments ="")
        createShardedPointAnnotation(points,img_dir)

    else:
        imgfile  = imgfiles[section]
        image = readSectionTif(imgfile)
        print("Detecting Cells")
        cells = get_cell_locations(image, intensity_threshold=threshold)
        viewer = ng_SingleSectionLocalViewer(image, cells)
        webbrowser.open(viewer.get_viewer_url())
        input("Press any key to continue")
        viewer = ng_SingleSectionLocalViewer(image, cells, viewer)
        webbrowser.open(viewer.get_viewer_url())
        input("Press any key to exit")


In [ ]:
import json
import urllib.parse


def to_url(path, param):
    # If no parameters are passed, use default settings
    # TODO: Determine param format
    if not param:
        param = {
            'starting_position': [0.0000013717664160139975,
                                  4307.64990234375,
                                  5821.93310546875],
            'x_mm': 1.25,
            'y_mm': 1.25,
            'z_mm': 50,
            'projection_scale': 13107.2,
            'cross_section_scale': 16.068429538550138,
        }
    # Create grayscale template layer
    mm_dict = get_mm_dict(param["z_mm"],
                          param["y_mm"],
                          param["x_mm"])
    template_layer = get_template_layer(template_src=path,
                                        mm_dict=mm_dict,
                                        public_name="ome.zarr_neuroglancer")
    url = get_final_url(image_layer_list=[template_layer],
                        template_layer=template_layer,
                        starting_position=param['starting_position'],
                        mm_dict=mm_dict,
                        projection_scale=param['projection_scale'],
                        cross_section_scale=param['cross_section_scale'])
    print(url)
    return url

"""
class ProcessTupleOperator(UDFOperatorV2):

    @overrides
    def process_tuple(self, tuple_: Tuple, port: int) -> Iterator[Optional[TupleLike]]:
        # host = "https://kiwi1.ics.uci.edu/neuroglancer-ftp"
        # path = tuple_['ome_zarr_path'].replace("/workspace/texera/core/amber/user-resources/files/40", host)
        # url = to_url(path, None)
        ftp_pos = tuple_['ome_zarr_path'].replace("/srv/brain-images/", "")
        url = "https://neuroglancer-demo.appspot.com/#!%7B%22dimensions%22:%7B%22z%22:%5B0.05%2C%22m%22%5D%2C%22y%22:%5B0.00125%2C%22m%22%5D%2C%22x%22:%5B0.00125%2C%22m%22%5D%7D%2C%22position%22:%5B145.8433074951172%2C3059.48779296875%2C5833.75244140625%5D%2C%22crossSectionScale%22:24.02241726399886%2C%22projectionOrientation%22:%5B-0.14507155120372772%2C0.1366751343011856%2C0.03381236642599106%2C0.9793522953987122%5D%2C%22projectionScale%22:18598.1290980758%2C%22layers%22:%5B%7B%22type%22:%22image%22%2C%22source%22:%7B%22url%22:%22zarr://https://xulabtexera.anat.uci.edu/neuroglancer-ftp/" + ftp_pos + "%22%2C%22transform%22:%7B%22outputDimensions%22:%7B%22z%22:%5B0.05%2C%22m%22%5D%2C%22y%22:%5B0.00125%2C%22m%22%5D%2C%22x%22:%5B0.00125%2C%22m%22%5D%7D%2C%22inputDimensions%22:%7B%22z%22:%5B0.05%2C%22m%22%5D%2C%22y%22:%5B0.00125%2C%22m%22%5D%2C%22x%22:%5B0.00125%2C%22m%22%5D%7D%7D%7D%2C%22tab%22:%22source%22%2C%22opacity%22:0.4%2C%22shader%22:%22#uicontrol%20invlerp%20normalized%5Cnvoid%20main%28%29%20%7B%5Cn%20%20emitGrayscale%28normalized%28%29%29%3B%5Cn%7D%22%2C%22shaderControls%22:%7B%22normalized%22:%7B%22range%22:%5B0%2C2086%5D%7D%7D%2C%22name%22:%22ome.zarr_neuroglancer1%22%7D%5D%2C%22selectedLayer%22:%7B%22visible%22:true%2C%22layer%22:%22ome.zarr_neuroglancer1%22%7D%2C%22layout%22:%224panel%22%2C%22selection%22:%7B%7D%2C%22layerListPanel%22:%7B%22visible%22:true%7D%7D"
        yield {"url": url}
"""

def get_mm_dict(z_mm=0.00125*5, y_mm=0.00125, x_mm=0.00125):
    """
    Return a dict used to generate the sizes of voxels in mm.
    """
    return {"z": [float(z_mm*0.001), "m"],
            "y": [float(y_mm*0.001), "m"],
            "x": [float(x_mm*0.001), "m"]}


def get_template_layer(
        template_src,
        mm_dict,
        public_name="Template layer",
        is_uint=False):
    """
    Return a dict containing configuration parameters for a grayscale
    template image.

    Parameters
    ----------
    template_src:
        URL-like location of the template image data. Assumes it is formatted
        as an OME-ZARR dataset.

    range_max:
        The pixel intensity value that gets set to white in the grayscale image.

    public_name:
        Name of the template image that will appear in the visualization

    is_uint:
        Set to true if the image data is an unsigned integer (that has some
        implications for the shader code used to control the color of
        the image)

    Returns
    -------
    Dict of configuration parameters for the grayscale template image
    """
    result = dict()
    result["type"] = "image"
    # result["source"] = f"zarr://{template_src}"
    result["source"] = {"url": f"zarr://{template_src}",
                        "transform": {
                            "outputDimensions": mm_dict,
                            "inputDimensions": mm_dict
                        }}
    result["blend"] = "default"
    # result["shader"] = get_grayscale_shader_code(
    #                       transparent=False,
    #                       range_max=range_max,
    #                       is_uint=is_uint)
    result["shader"] = get_simple_grayscale_shader_code()
    result["opacity"] = 0.4
    result["visible"] = True
    result["name"] = public_name
    return result


def get_grayscale_shader_code(
        transparent=True,
        range_max=20.0,
        threshold=0.0,
        is_uint=False):
    """
    Get shader code for a grayscale image.

    Parameters
    ----------
    transparent:
        If True, voxels lower than threshold are transparent.
        If False, voxels lower than threshold are black.

    range_max:
        Value that gets mapped to the maximum color intensity

    threshold:
        Value below which voxel is either transparent or black.

    is_uint:
        Set to true if the image data is an unsigned integer (that has some
        implications for the shader code used to control the color of
        the image)

    Returns
    -------
    A string representing the shader code for coloring the image.


    """
    if transparent:
        default = 'emitTransparent()'
    else:
        default = 'emitRGB(vec3(0, 0, 0))'

    code = f"#uicontrol invlerp normalized(range=[0,{range_max}])\n"
    code += "void main()"
    code += " {\n  "
    if is_uint:
        code += f"    if(int(getDataValue(0).value)>{int(threshold)})"
    else:
        code += f"    if(getDataValue(0)>{threshold})"
    code += "{\n"
    code += "        emitGrayscale(normalized())"
    code += ";\n}"
    code += "    else{\n"
    code += f"{default}"
    code += ";}\n}"
    return code


def get_simple_grayscale_shader_code():
    """
    Get shader code for a grayscale image.

    Returns
    -------
    A string representing the shader code for coloring the image.
    """
    code = f"#uicontrol invlerp normalized\n"
    code += "void main()"
    code += " {\n  "
    code += "emitGrayscale(normalized());\n}"
    return code


def get_base_url():
    """
    Return the base URL of the neuroglancer instance that will be running
    your visualization. Note: most of the work is actually done on the client
    browser.
    """
    return "https://neuroglancer-demo.appspot.com/#!"


def json_to_url(json_data):
    """
    Serialize the config dict into an HTML-compliant string.
    """
    return urllib.parse.quote(json_data)


def get_final_url(
        image_layer_list,
        template_layer=None,
        segmentation_layer=None,
        starting_position=None,
        mm_dict=None,
        projection_scale=2048,
        cross_section_scale=2.6):
    """
    Create a valid neuroglancer URL

    Parameters
    ----------
    image_layer_list:
        A list of dicts. Each dict is the configuration for a different
        image layer.

    template_layer:
        A dict containing the configuration for the template layer.
        (if None, not included)

    segmentation_layer:
        A dict containing the configuration for the segmentation layer.
        (if None, not included)

    starting_position:
        Optional (x, y, z) tuple of the voxel on which you want the
        visualization to automatically load. If None, will use
        neuroglancer's default.

    x_mm, y_mm, z_mm:
        (x, y, z) sizes of voxels in millimeters

    projection_scale, cross_section_scale:
        Scaling parameters for  visualization
        (not actually sure what they represent)

    Returns
    -------
    A string containing a URL to a neuroglancer visualization
    of the image layers you input.
    """

    if not isinstance(image_layer_list, list):
        image_layer_list = [image_layer_list]

    url = get_base_url()

    layer_list = image_layer_list
    if template_layer is not None:
        layer_list.append(template_layer)
    if segmentation_layer is not None:
        layer_list.append(segmentation_layer)
    if mm_dict is None:
        mm_dict = get_mm_dict()

    layers = dict()
    layers["dimensions"] = mm_dict
    layers["crossSectionScale"] = cross_section_scale
    layers["projectionScale"] = projection_scale
    layers["layers"] = layer_list
    layers["selectedLayer"] = {"visible": True, "layer": "new layer"}
    layers["layout"] = "4panel"

    if starting_position is not None:
        layers["position"] = [float(x) for x in starting_position]
    url = f"{url}{json_to_url(json.dumps(layers))}"

    return url

In [8]:
import numpy as np

output_indices = np.load("outputIndices.npy")
output_indices = output_indices[~np.all(output_indices == [0, 0, 0], axis=1)]

print(output_indices)

np.savetxt("outputIndices.txt", output_indices , "%d %d %d", 
           header = "index\n" + str(output_indices.shape[0]), 
           comments = "")

[[518  97 167]
 [503 103 196]
 [527 296 224]
 ...
 [158 197 230]
 [158 199 234]
 [158 201 231]]


In [7]:
output_indices.shape

(3016, 3)